<a href="https://colab.research.google.com/github/byshadowoz/CCFDM/blob/main/nose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model of Detection of credit card fraud 

In [ ]:
#instaling library pandas
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
#importing pandas and numpy library 

# Raw data

In [ ]:
data = pd.read_csv('fraudtest.csv', index_col=0)
data.info()
# taking the csv from de archives(download in kaggle.com link in github)

NameError: name 'pd' is not defined

# Processing data
### this will create a new colum named Time wich is the time between the first row of trans_date_trans_time minus all the rows divided in 60. then removes the useful columns of the dataframe. the last is dumies wich make the string rows in columns with boolean values 

In [3]:
coluseless = ['first','last','gender','street','lat','long','zip','city_pop','dob','trans_num','unix_time','trans_date_trans_time','merch_lat','merch_long']

def processingDT(df):
  #creating new column with the time btween the time in the column and the first column (first column = 0)
  if 'trans_date_trans_time' in df.columns:
    df.trans_date_trans_time = pd.to_datetime(df['trans_date_trans_time'])
    df['Time'] = (df['trans_date_trans_time'] - df['trans_date_trans_time'].iloc[0]).dt.total_seconds()
    df['Time'].fillna(0, inplace=True)
    df['Time'] = df['Time'] / 60

    #drop unuseful data for the ML model
    coluseless = ['first','last','gender','street','lat','long','zip','city_pop','dob','trans_num','unix_time','trans_date_trans_time','merch_lat','merch_long']
    for col in coluseless:
        if col in df.columns:
          df.drop(col,axis=1, inplace=True)
  

  #Processing string data to boolean data
  def dummies(df,columns):

    dummies = pd.get_dummies(df[columns])

    df = pd.concat([df, dummies], axis=1)

    df.drop(columns=columns, inplace=True)

    return df

  df = dummies(df,[ 'merchant', 'category','city', 'state', 'job',])

  return df

In [56]:
data = processingDT(data)

/tmp/ipykernel_8448/1835225305.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time'].fillna(0, inplace=True)


# Procesing for modeling

In [ ]:
x_cols =list(data.columns)
x_cols.remove('is_fraud')
x_cols.remove('Time')
x_cols.remove('amt')

## divided the model in two and grab the most low dataframe and then add all the values wich the column is_fraud is True = 1

In [12]:
data_part1_2 = data.head(138929)
data_part1_3 = data.tail(416790)

In [13]:
data_part1_3 = data_part1_3[data_part1_3.is_fraud == 1]

In [14]:
data_part1_3 = data_part1_3.reset_index(drop=True)

In [15]:
data_part = pd.concat([data_part1_2,data_part1_3])

In [17]:
data_part

,cc_num,amt,is_fraud,Time,merchant_fraud_Abbott-Rogahn,merchant_fraud_Abbott-Steuber,merchant_fraud_Abernathy and Sons,merchant_fraud_Abshire PLC,"merchant_fraud_Adams, Kovacek and Kuhlman",merchant_fraud_Adams-Barrows,...,job_Video editor,job_Visual merchandiser,job_Volunteer coordinator,job_Warden/ranger,job_Waste management officer,job_Water engineer,job_Water quality scientist,job_Web designer,job_Wellsite geologist,job_Writer
0,2.291160e+15,2.86,0,0.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,3.573030e+15,29.84,0,0.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3.598220e+15,41.28,0,0.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3.591920e+15,60.05,0,1.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3.526830e+15,3.19,0,1.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2.242180e+15,1041.51,1,265551.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1523,2.242180e+15,868.09,1,265564.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1524,2.242180e+15,1039.42,1,265577.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1525,2.242180e+15,289.27,1,265612.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Modeling

### PCA and StandardScaler are used for zip the 2088 columns

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()
data_part[x_cols] = ss.fit_transform(data_part[x_cols])

In [ ]:
pca2 = PCA(n_components=28, random_state=42)
pca2 = pca2.fit_transform(data_part[x_cols])

In [ ]:
#the name of columns that are compressed
columns = ['V' + str(i) for i in range(1, 29)]

In [ ]:
data_part = data_part.reset_index(drop=True)

In [ ]:
data2 = pd.DataFrame(pca2, columns=columns).assign(is_fraud=data_part['is_fraud'],Time=data_part['Time'],Amount=data_part['amt'])

In [ ]:
data2[data2.is_fraud == 1]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,is_fraud,Time,Amount
1685,-0.598172,1.144613,-0.094510,0.859783,-1.011711,0.273199,0.727060,-1.906297,-0.736238,-0.001437,...,-0.443760,-0.490428,-0.030474,1.545240,0.434241,-0.478522,0.463882,1,592.0,24.84
1767,-1.480529,0.222242,1.295261,1.365133,2.788950,-2.822044,2.669732,0.585923,-1.853767,0.483757,...,-3.983478,-1.416715,0.104810,2.107172,-2.274064,-0.167767,1.985643,1,618.0,780.52
1781,-1.271606,0.120356,1.465947,1.133249,2.766102,-2.587641,2.578613,0.505140,-1.745468,0.100368,...,-3.687705,-1.451466,0.204187,2.378236,-2.394648,-0.018352,2.039806,1,623.0,620.33
1784,1.743642,-1.296773,0.433705,-3.781448,-2.790281,-0.258579,1.645773,-3.878941,-2.384804,0.102737,...,-0.120302,0.148986,-2.109904,-0.837889,1.047046,-2.220008,-1.334460,1,624.0,1077.69
1857,-0.815687,1.383508,-0.072413,0.736195,-0.783435,0.417599,0.575419,-2.198089,-0.785825,-0.297522,...,-0.337583,-0.478347,0.136176,1.477905,0.481226,-0.645979,0.451778,1,648.0,842.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140451,-1.885197,-1.901633,-0.328577,0.413878,-1.585098,0.579325,0.020023,0.070661,1.485562,0.856125,...,-1.021674,0.118913,-2.673635,-2.147107,-1.147776,0.033870,-2.094554,1,265551.0,1041.51
140452,-1.884568,-1.865341,-0.385483,0.385217,-1.621335,0.595049,-0.018489,0.083034,1.475932,0.945927,...,-1.062314,0.022884,-2.721337,-2.219702,-1.177209,0.053716,-2.214453,1,265564.0,868.09
140453,-1.791368,-1.933004,-0.455697,0.551157,-1.971012,0.426638,-0.012275,0.414370,1.585215,1.286257,...,-1.057889,0.028093,-2.760386,-2.211881,-1.059834,0.244320,-1.994913,1,265577.0,1039.42
140454,-1.917875,-1.994005,-0.424867,0.592209,-1.757509,0.562001,0.002287,-0.145944,1.640667,0.913690,...,-0.889942,-0.056516,-2.833079,-2.060295,-0.937546,0.249133,-1.997296,1,265612.0,289.27


# Modeling Logistic Regresion and seeing acuraccy

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
x = data2.drop(columns=['is_fraud'])
y = data2.is_fraud

In [ ]:
model = LogisticRegression()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
model.fit(x_train, y_train)

LogisticRegression()

In [ ]:
prediction = model.predict(x_test)

In [ ]:
prediction_prof = pd.DataFrame({"prediction": prediction, "real": y_test})

In [ ]:
prediction_prof.sum()

prediction    409
real          642
dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, prediction)

print(cm)

[[41474    21]
 [  254   388]]


#### in general the acuraccy score is 99.34% but just in the fraud cases is 60.43% of acuraccy

# importing Model

In [ ]:
import pickle

with open('CCFD.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved CCFD.pkl")

Modelo guardado como CCFD.pkl


In [ ]:
from google.colab import files

files.download('CCFD.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Creating CSV for db table

In [4]:
dataCSV = pd.read_csv('fraudtestdumed.csv')

In [5]:
dataCSV = processingDT(dataCSV)

/tmp/ipykernel_64117/686612443.py:6: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.trans_date_trans_time = pd.to_datetime(df['trans_date_trans_time'])
/tmp/ipykernel_64117/686612443.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time'].fillna(0, inplace=True)


In [7]:
dataCSV.drop(dataCSV.index, inplace=True)

print(dataCSV)

Empty DataFrame
Columns: [cc_num, amt, is_fraud, Time, merchant_fraud_Abbott-Rogahn, merchant_fraud_Abbott-Steuber, merchant_fraud_Abernathy and Sons, merchant_fraud_Abshire PLC, merchant_fraud_Adams, Kovacek and Kuhlman, merchant_fraud_Adams-Barrows, merchant_fraud_Altenwerth, Cartwright and Koss, merchant_fraud_Altenwerth-Kilback, merchant_fraud_Ankunding LLC, merchant_fraud_Ankunding-Carroll, merchant_fraud_Armstrong, Walter and Gottlieb, merchant_fraud_Auer LLC, merchant_fraud_Auer-Mosciski, merchant_fraud_Auer-West, merchant_fraud_Bahringer Group, merchant_fraud_Bahringer, Bergnaum and Quitzon, merchant_fraud_Bahringer, Osinski and Block, merchant_fraud_Bahringer, Schoen and Corkery, merchant_fraud_Bahringer-Larson, merchant_fraud_Bahringer-Streich, merchant_fraud_Bailey-Morar, merchant_fraud_Balistreri-Nader, merchant_fraud_Barrows PLC, merchant_fraud_Bartoletti and Sons, merchant_fraud_Bartoletti-Wunsch, merchant_fraud_Barton Inc, merchant_fraud_Barton LLC, merchant_fraud_Bashir

In [8]:
dataCSV.to_csv('fraudtestdumed.csv',index=False)